In [1]:
import pandas as pd
import psycopg
import mlflow
import os

In [2]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20241219_a60d0b01a0",
    "user": "mle_20241219_a60d0b01a0",
    "password": "e1b4b5c40ab84dcc8dfe8b926160a092",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [3]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

    # обновите словарь stats
    for key, value in column_stat.items():
        stats[key] = value


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке

In [4]:
stats

{'type_Month-to-month': 3875,
 'type_Two year': 1695,
 'type_One year': 1473,
 'paperless_billing_Yes': 4171,
 'paperless_billing_No': 2872,
 'internet_service_Fiber optic': 3096,
 'internet_service_DSL': 2421,
 'online_security_No': 3498,
 'online_security_Yes': 2019,
 'online_backup_No': 3088,
 'online_backup_Yes': 2429,
 'device_protection_No': 3095,
 'device_protection_Yes': 2422,
 'tech_support_No': 3473,
 'tech_support_Yes': 2044,
 'streaming_tv_No': 2810,
 'streaming_tv_Yes': 2707,
 'streaming_movies_No': 2785,
 'streaming_movies_Yes': 2732,
 'gender_Male': 3555,
 'gender_Female': 3488,
 'senior_citizen_0': 5901,
 'senior_citizen_1': 1142,
 'partner_No': 3641,
 'partner_Yes': 3402,
 'dependents_No': 4933,
 'dependents_Yes': 2110,
 'multiple_lines_No': 3390,
 'multiple_lines_Yes': 2971,
 'target_0': 5174,
 'target_1': 1869,
 'data_length': 7043,
 'monthly_charges_min': 18.25,
 'monthly_charges_max': 118.75,
 'monthly_charges_mean': 64.76169246059918,
 'monthly_charges_median': 70

In [23]:
!export $(cat .env | xargs)

In [24]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

EXPERIMENT_NAME = "churn_pmm"
RUN_NAME = "data_check"

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(
        EXPERIMENT_NAME, 
        artifact_location="s3://s3-student-mle-20241219-a60d0b01a0"
    )
else:
    experiment_id = experiment.experiment_id


with mlflow.start_run(run_name=EXPERIMENT_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id 
    
    # логируем метрики эксперимента
    for metric, value in stats.items():
        mlflow.log_metric(metric, value)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    with open('columns.txt', 'w', encoding='utf-8') as fd:
        fd.write(','.join(df.columns.tolist()))
        
    df.to_csv("users_churn.csv")

    mlflow.log_artifact("columns.txt") 
    mlflow.log_artifact("users_churn.csv") 

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) 
status = run.info.status

# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
assert status == 'FINISHED' 


# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы
os.remove('columns.txt') 
os.remove("users_churn.csv") 